<a href="https://colab.research.google.com/github/sashwot10/5CS037/blob/main/worksheet8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris, load_wine
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error

class CustomDecisionTree:
    def __init__(self, max_depth=None):
        """Initializes the tree with a specified maximum depth[cite: 50, 163]."""
        self.max_depth = max_depth
        self.tree = None

    def _entropy(self, y):
        """Calculates entropy using H = -sum(p * log2(p))[cite: 208, 217]."""
        if len(y) == 0: return 0
        class_probs = np.bincount(y) / len(y)
        # Add epsilon (1e-9) to avoid log(0) [cite: 139, 218]
        return -np.sum([p * np.log2(p + 1e-9) for p in class_probs if p > 0])

    def _information_gain(self, parent, left, right):
        """Computes reduction in entropy after a split[cite: 107, 197]."""
        parent_entropy = self._entropy(parent)
        n = len(parent)
        weighted_child_entropy = (len(left) / n) * self._entropy(left) + (len(right) / n) * self._entropy(right)
        return parent_entropy - weighted_child_entropy

    def _build_tree(self, X, y, depth=0):
        """Recursively builds the tree by finding the best split[cite: 63, 177]."""
        num_samples, num_features = X.shape
        unique_classes = np.unique(y)

        # Stopping conditions: pure node or max depth reached [cite: 74, 191]
        if len(unique_classes) == 1 or (self.max_depth and depth >= self.max_depth) or num_samples <= 1:
            return {'class': np.bincount(y).argmax()}

        best_gain = -1
        best_split = None

        for feature_idx in range(num_features):
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                left_mask = X[:, feature_idx] <= threshold
                right_mask = ~left_mask

                if np.any(left_mask) and np.any(right_mask):
                    gain = self._information_gain(y, y[left_mask], y[right_mask])
                    if gain > best_gain:
                        best_gain = gain
                        best_split = {
                            'feature_idx': feature_idx,
                            'threshold': threshold,
                            'left_mask': left_mask,
                            'right_mask': right_mask
                        }

        if best_split is None:
            return {'class': np.bincount(y).argmax()}

        # Recursive step [cite: 101, 195]
        left_tree = self._build_tree(X[best_split['left_mask']], y[best_split['left_mask']], depth + 1)
        right_tree = self._build_tree(X[best_split['right_mask']], y[best_split['right_mask']], depth + 1)

        return {
            'feature_idx': best_split['feature_idx'],
            'threshold': best_split['threshold'],
            'left_tree': left_tree,
            'right_tree': right_tree
        }

    def fit(self, X, y):
        """Trains the model[cite: 55, 171]."""
        self.tree = self._build_tree(X, y)

    def _predict_single(self, x, tree):
        """Traverses the tree for a single sample[cite: 146, 226]."""
        if 'class' in tree:
            return tree['class']
        if x[tree['feature_idx']] <= tree['threshold']:
            return self._predict_single(x, tree['left_tree'])
        else:
            return self._predict_single(x, tree['right_tree'])

    def predict(self, X):
        """Predicts labels for a matrix X[cite: 137, 220]."""
        return [self._predict_single(x, self.tree) for x in X]

In [4]:
# Load and Split data [cite: 236]
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

# Custom Tree [cite: 252]
custom_tree = CustomDecisionTree(max_depth=3)
custom_tree.fit(X_train, y_train)
y_pred_custom = custom_tree.predict(X_test)
acc_custom = accuracy_score(y_test, y_pred_custom)

# Scikit-Learn Tree [cite: 263]
sk_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
sk_tree.fit(X_train, y_train)
y_pred_sk = sk_tree.predict(X_test)
acc_sk = accuracy_score(y_test, y_pred_sk)

print(f"Custom Tree Accuracy: {acc_custom:.4f}")
print(f"Scikit-learn Accuracy: {acc_sk:.4f}")

Custom Tree Accuracy: 1.0000
Scikit-learn Accuracy: 1.0000


In [6]:
# Load Wine Dataset [cite: 281]
wine = load_wine()
X_w_train, X_w_test, y_w_train, y_w_test = train_test_split(wine.data, wine.target, test_size=0.2, random_state=42)

# Random Forest Classifier & Tuning [cite: 283, 286]
rf_clf = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(rf_clf, param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_w_train, y_w_train)

best_rf = grid_search.best_estimator_
y_pred_rf = best_rf.predict(X_w_test)
print(f"Best RF F1 Score: {f1_score(y_w_test, y_pred_rf, average='macro'):.4f}")

Best RF F1 Score: 1.0000


In [7]:
# Random Forest Regressor & Tuning [cite: 290, 291]
rf_reg = RandomForestRegressor(random_state=42)
param_dist = {
    'n_estimators': [100, 300, 500],
    'max_features': ['sqrt', 'log2', None],
    'min_samples_leaf': [1, 2, 4]
}

random_search = RandomizedSearchCV(rf_reg, param_dist, n_iter=5, cv=5, random_state=42)
random_search.fit(X_w_train, y_w_train)

print(f"Best Regression Parameters: {random_search.best_params_}")

Best Regression Parameters: {'n_estimators': 100, 'min_samples_leaf': 1, 'max_features': 'log2'}
